In [1]:
%%js
Jupyter.notebook.get_cell(0).element.hide()
Jupyter.notebook.get_cell(1).element.hide()
Jupyter.notebook.get_cell(25).element.hide()
Jupyter.notebook.get_cell(26).element.hide()

<IPython.core.display.Javascript object>

In [2]:

import pandas as pd


# Annual Prediction 

In [3]:
train2 = pd.read_csv("Training 2020.csv")

In [4]:
train1 = pd.read_csv("Training 2021.csv")

In [5]:
test = pd.read_csv("Testpred 2022.csv")

In [6]:
# returns the number of items in an object
train2_len = len(train2)
train1_len = len(train1)
test_len = len(test)

In [7]:
# concatinate the objects 
df = pd.concat([train2, train1, test], axis=0)
pd.set_option('display.max_rows', df.shape[0]+1)
df.reset_index(drop=True, inplace=True)
df

,Name,Size,Cost,Category,Sales,Year
0,Kamias,Medium,100,Tree,3400.0,2020
1,Kalamasi,Medium,120,Tree,4080.0,2020
2,Papaya,Small,50,Tree,1700.0,2020
3,Tundan,Small,80,Tree,3200.0,2020
4,Lakatan,Small,150,Tree,10500.0,2020
5,Rambutan,Medium,250,Tree,5000.0,2020
6,Suha,Small,130,Tree,1950.0,2020
7,Atis,Medium,130,Tree,3770.0,2020
8,Guyabano,Medium,100,Tree,2200.0,2020
9,Kamias,Medium,100,Tree,3700.0,2021


In [8]:
%%capture
print(len(train2), len(train1), len(test), len(df))

In [9]:
%%capture
# get unique values x serves as a placeholder for the value
df.apply(lambda x: len(x.unique()))

In [10]:
%%capture
df.isnull().sum()

In [11]:
%%capture
# combines object types
cat_col = []
for x in df.dtypes.index:
    if df.dtypes[x] == 'object':
        cat_col.append(x)
cat_col

In [12]:
%%capture
cat_col.remove('Name')
cat_col

In [13]:
%%capture
# counts the value in objects
for col in cat_col:
    print(col)
    print(df[col].value_counts())
    print();

In [14]:
%%capture
# creates values and column for objects
size_mode = df.pivot_table(values = 'Size', columns = 'Category', aggfunc=(lambda x: x.mode()[0]))
size_mode

In [15]:
miss_bool = df['Size'].isnull()
df.loc[miss_bool, 'Size'] = df.loc[miss_bool, 'Category'].apply(lambda x: size_mode[x])

In [16]:
%%capture
df['Size'].isnull().sum()

In [17]:
df['Total_Year'] = 2022 - df['Year']

In [18]:
%%capture
df['Total_Year']

In [19]:
%%capture
# creates entry for the column
df = pd.get_dummies(df, columns=[ 'Size', 'Category'])
df.head();

In [20]:
%%capture
df.tail();

In [21]:
%%capture
train2 = df.iloc[:train2_len,:]
train1 = df.iloc[train2_len:,:]
test = df.iloc[train2_len:,:]

In [22]:
%%capture
test = test.drop(columns=['Sales'])
test.head()

In [23]:
%%capture
test.tail();

In [24]:
# preparing for model fitting
X = train2.drop(columns=['Year', 'Name', 'Sales'])
y = train2['Sales']

In [25]:
from sklearn.metrics import mean_squared_error
def train2(model, X, y):
    model.fit(X, y)
    model.score(X, y)
    

In [26]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
train2(model, X, y)
coef = pd.Series(model.feature_importances_, X.columns).sort_values(ascending=False)

In [27]:

%%capture
x_test = test.drop(columns=['Year', 'Name'])
x_test.head();

# Prediction

In [28]:
class RandomForestRegresor:
    def __init__(self, n_trees=10, max_depth=3, min_samples_split=2, n_feature=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_feature
        self.trees = []

    def fit(self, X, y): 
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)
        
    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        return predictions


In [29]:
pred = model.predict(X)
print("Model Report Score Accuracy:", model.score(X, y))

Model Report Score Accuracy: 0.7581884220321887


In [30]:
pred = model.predict(x_test)
pred

array([3009.4, 3949. , 2184.5, 2769.9, 7249.8, 6123.7, 2798.2, 3728. ,
       3009.4, 3009.4, 3949. , 2184.5, 2769.9, 7249.8, 6123.7, 2798.2,
       3728. , 3009.4])

# Annual Prediction result

In [31]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred
mission['Year'] = 2022
pd.set_option('display.max_rows', df.shape[0]+1)
mission


,Name,sales,Year
9,Kamias,3009.4,2022
10,Kalamasi,3949.0,2022
11,Papaya,2184.5,2022
12,Tundan,2769.9,2022
13,Lakatan,7249.8,2022
14,Rambutan,6123.7,2022
15,Suha,2798.2,2022
16,Atis,3728.0,2022
17,Guyabano,3009.4,2022
18,Kamias,3009.4,2022


# Quarterly Prediction

In [32]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred / 12
mission['Quarterly'] = 3
pd.set_option('display.max_rows', df.shape[0]+1)
mission

,Name,sales,Quarterly
9,Kamias,250.783333,3
10,Kalamasi,329.083333,3
11,Papaya,182.041667,3
12,Tundan,230.825000,3
13,Lakatan,604.150000,3
14,Rambutan,510.308333,3
15,Suha,233.183333,3
16,Atis,310.666667,3
17,Guyabano,250.783333,3
18,Kamias,250.783333,3


# Monthly Prediction

In [33]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred / 36
mission['monthly'] = 1
pd.set_option('display.max_rows', df.shape[0]+1)
mission

,Name,sales,monthly
9,Kamias,83.594444,1
10,Kalamasi,109.694444,1
11,Papaya,60.680556,1
12,Tundan,76.941667,1
13,Lakatan,201.383333,1
14,Rambutan,170.102778,1
15,Suha,77.727778,1
16,Atis,103.555556,1
17,Guyabano,83.594444,1
18,Kamias,83.594444,1
